In [2]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV

In [3]:
%run data.ipynb

In [4]:
%run processing_methods.ipynb

In [5]:
%run ml_methods.ipynb

In [6]:
x_ = x_all.copy()
y_ = y_all.copy()

In [33]:
x_trainval, x_test, y_trainval, y_test = train_test_split(x_,y_)
x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)

In [7]:
data_sets = []
best_scores = []
test_scores = []
results = []
models = []
for i in range(20):
    
    x_trainval, x_test, y_trainval, y_test = train_test_split(x_,y_)
    x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)
    data_sets.append([x_trainval, x_test, y_trainval, y_test, x_train, x_val, y_train, y_val])
    
    best_score = 0
    for C in [0.01, 0.1, 1, 10,50, 100, 200, 500, 1000]:
        
        model = LinearSVC(C=C, max_iter=10000000,dual = True).fit(x_train, y_train)
        score = model.score(x_val, y_val)
        if score > best_score:
            best_score = score
            best_parameters = {'C': C}

    model  = LinearSVC(C=best_parameters['C'], max_iter=10000000).fit(x_trainval, y_trainval)
    test_score = model.score(x_test, y_test)
    results.append(best_parameters)
    test_scores.append(test_score)
    best_scores.append(best_score)
    models.append(model)

In [8]:
for i in range(20):
    print("{}: ".format(i)+"best_parametrs: {}, ".format(results[i])+"test_score: {}, ".format(test_scores[i])+"best_score: {}, ".format(best_scores[i]))

0: best_parametrs: {'C': 10}, test_score: 0.8771929824561403, best_score: 0.9224806201550387, 
1: best_parametrs: {'C': 50}, test_score: 0.8596491228070176, best_score: 0.9302325581395349, 
2: best_parametrs: {'C': 1000}, test_score: 0.9122807017543859, best_score: 0.8914728682170543, 
3: best_parametrs: {'C': 50}, test_score: 0.9005847953216374, best_score: 0.9069767441860465, 
4: best_parametrs: {'C': 50}, test_score: 0.8947368421052632, best_score: 0.9069767441860465, 
5: best_parametrs: {'C': 100}, test_score: 0.9473684210526315, best_score: 0.9302325581395349, 
6: best_parametrs: {'C': 10}, test_score: 0.8713450292397661, best_score: 0.9224806201550387, 
7: best_parametrs: {'C': 1000}, test_score: 0.935672514619883, best_score: 0.9302325581395349, 
8: best_parametrs: {'C': 1000}, test_score: 0.9415204678362573, best_score: 0.9147286821705426, 
9: best_parametrs: {'C': 1000}, test_score: 0.8947368421052632, best_score: 0.9147286821705426, 
10: best_parametrs: {'C': 50}, test_score:

In [9]:
c_col = []
for i in range(20):
    c_col.append(results[i]['C'])
c_col = pd.Series(np.array(c_col))
test_col = pd.Series(test_scores)
best_col = pd.Series(best_scores)
res = pd.concat([c_col,best_col,test_col],axis=1)
res.to_excel('results/svc_general.xlsx')

In [10]:
mean = test_col.mean()
print("Mean: {},".format(mean))

Mean: 0.9119883040935672,


In [11]:
mode = c_col.mode()
mode[0]

50

In [12]:
# best test score
iter = test_col.idxmax()
y_pred = models[iter].predict(data_sets[iter][1])
y_test = data_sets[iter][3]
classification_report_ = classification_report(y_test,y_pred,output_dict=True)
classification_report_ = pd.DataFrame(classification_report_)
classification_report_

,0.0,1.0,2.0,accuracy,macro avg,weighted avg
precision,0.970588,0.842105,1.000000,0.959064,0.937564,0.961599
recall,1.000000,0.941176,0.727273,0.959064,0.889483,0.959064
f1-score,0.985075,0.888889,0.842105,0.959064,0.905356,0.957119
support,132.000000,17.000000,22.000000,0.959064,171.000000,171.000000


In [13]:
confusion_matrix_ = confusion_matrix(y_test,y_pred)
confusion_matrix_ = pd.DataFrame(confusion_matrix_)
confusion_matrix_

,0,1,2
0,132,0,0
1,1,16,0
2,3,3,16


In [14]:
# worst test score
iter = test_col.idxmin()
y_pred = models[iter].predict(data_sets[iter][1])
y_test = data_sets[iter][3]
classification_report_ = classification_report(y_test,y_pred,output_dict=True)
classification_report_ = pd.DataFrame(classification_report_)
classification_report_

,0.0,1.0,2.0,accuracy,macro avg,weighted avg
precision,0.900000,0.750000,0.705882,0.859649,0.785294,0.853612
recall,0.943548,0.666667,0.600000,0.859649,0.736738,0.859649
f1-score,0.921260,0.705882,0.648649,0.859649,0.758597,0.855369
support,124.000000,27.000000,20.000000,0.859649,171.000000,171.000000


In [15]:
confusion_matrix_ = confusion_matrix(y_test,y_pred)
confusion_matrix_ = pd.DataFrame(confusion_matrix_)
confusion_matrix_

,0,1,2
0,117,3,4
1,8,18,1
2,5,3,12


In [8]:
scores_C = []
results_C = []
y_pred = 0
best_score = 0
for i in range(20):
    
    x_trainval, x_test, y_trainval, y_test = train_test_split(x_,y_)
    x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)
      
    model  = LogisticRegression(C=50, max_iter=100000).fit(x_trainval, y_trainval)
    score = model.score(x_test, y_test)
    if score > best_score:
        y_pred = model.predict(x_test)
        confusion_matrix_ = confusion_matrix(y_test,y_pred)
        classification_report_ = classification_report(y_test,y_pred,output_dict=True)
    scores_C.append(score)

In [11]:
for i in range(20):
    print("{}: ".format(i)+"C: {}, ".format(50)+"test_score: {}, ".format(scores_C[i]))

0: C: 50, test_score: 0.8888888888888888, 
1: C: 50, test_score: 0.8596491228070176, 
2: C: 50, test_score: 0.8713450292397661, 
3: C: 50, test_score: 0.9239766081871345, 
4: C: 50, test_score: 0.8947368421052632, 
5: C: 50, test_score: 0.8830409356725146, 
6: C: 50, test_score: 0.8830409356725146, 
7: C: 50, test_score: 0.8947368421052632, 
8: C: 50, test_score: 0.8771929824561403, 
9: C: 50, test_score: 0.8947368421052632, 
10: C: 50, test_score: 0.8713450292397661, 
11: C: 50, test_score: 0.9064327485380117, 
12: C: 50, test_score: 0.9122807017543859, 
13: C: 50, test_score: 0.8830409356725146, 
14: C: 50, test_score: 0.9064327485380117, 
15: C: 50, test_score: 0.9122807017543859, 
16: C: 50, test_score: 0.8830409356725146, 
17: C: 50, test_score: 0.8771929824561403, 
18: C: 50, test_score: 0.935672514619883, 
19: C: 50, test_score: 0.8947368421052632, 


In [14]:
scores_C = pd.Series(scores_C)
res_C = pd.concat([scores_C],axis=1)
res_C.to_excel('results/svc_C.xlsx')

In [15]:
test_col = pd.Series(scores_C)
test_col.max()

0.935672514619883

In [16]:
test_col.idxmax()

18

In [17]:
confusion_matrix_ = pd.DataFrame(confusion_matrix_)
confusion_matrix_

,0,1,2
0,120,1,3
1,4,22,2
2,6,2,11


In [18]:
classification_report_df = pd.DataFrame(classification_report_)
classification_report_df

,0.0,1.0,2.0,accuracy,macro avg,weighted avg
precision,0.923077,0.880000,0.687500,0.894737,0.830192,0.889848
recall,0.967742,0.785714,0.578947,0.894737,0.777468,0.894737
f1-score,0.944882,0.830189,0.628571,0.894737,0.801214,0.890956
support,124.000000,28.000000,19.000000,0.894737,171.000000,171.000000


In [19]:
scores_C = []
results_C = []
y_pred = 0
best_score = 0
for i in range(20):
    
    x_trainval, x_test, y_trainval, y_test = train_test_split(x_,y_)
    x_train, x_val, y_train, y_val = train_test_split(x_trainval,y_trainval)
      
    model  = LogisticRegression(C=1000, max_iter=100000).fit(x_trainval, y_trainval)
    score = model.score(x_test, y_test)
    if score > best_score:
        y_pred = model.predict(x_test)
        confusion_matrix_ = confusion_matrix(y_test,y_pred)
        classification_report_ = classification_report(y_test,y_pred,output_dict=True)
    scores_C.append(score)

In [20]:
for i in range(20):
    print("{}: ".format(i)+"C: {}, ".format(1000)+"test_score: {}, ".format(scores_C[i]))

0: C: 1000, test_score: 0.8713450292397661, 
1: C: 1000, test_score: 0.9005847953216374, 
2: C: 1000, test_score: 0.935672514619883, 
3: C: 1000, test_score: 0.9122807017543859, 
4: C: 1000, test_score: 0.9064327485380117, 
5: C: 1000, test_score: 0.9005847953216374, 
6: C: 1000, test_score: 0.8888888888888888, 
7: C: 1000, test_score: 0.8888888888888888, 
8: C: 1000, test_score: 0.9239766081871345, 
9: C: 1000, test_score: 0.847953216374269, 
10: C: 1000, test_score: 0.9064327485380117, 
11: C: 1000, test_score: 0.8596491228070176, 
12: C: 1000, test_score: 0.9298245614035088, 
13: C: 1000, test_score: 0.8888888888888888, 
14: C: 1000, test_score: 0.9122807017543859, 
15: C: 1000, test_score: 0.9005847953216374, 
16: C: 1000, test_score: 0.9122807017543859, 
17: C: 1000, test_score: 0.935672514619883, 
18: C: 1000, test_score: 0.8771929824561403, 
19: C: 1000, test_score: 0.9239766081871345, 


In [21]:
scores_C = pd.Series(scores_C)
res_C = pd.concat([scores_C],axis=1)
res_C.to_excel('results/svc_C2.xlsx')

In [22]:
confusion_matrix_ = pd.DataFrame(confusion_matrix_)
confusion_matrix_

,0,1,2
0,125,0,4
1,3,19,1
2,4,1,14


In [23]:
classification_report_df = pd.DataFrame(classification_report_)
classification_report_df

,0.0,1.0,2.0,accuracy,macro avg,weighted avg
precision,0.946970,0.950000,0.736842,0.923977,0.877937,0.924030
recall,0.968992,0.826087,0.736842,0.923977,0.843974,0.923977
f1-score,0.957854,0.883721,0.736842,0.923977,0.859472,0.923326
support,129.000000,23.000000,19.000000,0.923977,171.000000,171.000000
